In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
import AFQ.registration as reg
import AFQ.utils.models as ut
import AFQ.data as afd
import AFQ.tractography as aft
import AFQ.csd as csd
import dipy.data as dpd
import nibabel as nib

In [3]:
img, gtab = dpd.read_stanford_hardi()

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/stanford_hardi 


In [4]:
afq_templates = afd.read_templates()

Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/AFQ_data/templates 


In [5]:
afq_templates

{'ATR_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b39c5c0>,
 'ATR_roi1_R': <nibabel.nifti1.Nifti1Image at 0x11b39c2b0>,
 'ATR_roi2_L': <nibabel.nifti1.Nifti1Image at 0x11b384f60>,
 'ATR_roi2_R': <nibabel.nifti1.Nifti1Image at 0x11b40c198>,
 'CGC_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b411b38>,
 'CGC_roi1_R': <nibabel.nifti1.Nifti1Image at 0x11b39c438>,
 'CGC_roi2_L': <nibabel.nifti1.Nifti1Image at 0x11b40c278>,
 'CGC_roi2_R': <nibabel.nifti1.Nifti1Image at 0x11b411208>,
 'CST_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b411828>,
 'CST_roi1_R': <nibabel.nifti1.Nifti1Image at 0x11b40ca20>,
 'CST_roi2_L': <nibabel.nifti1.Nifti1Image at 0x11b40c710>,
 'CST_roi2_R': <nibabel.nifti1.Nifti1Image at 0x11b40cd30>,
 'FA_L': <nibabel.nifti1.Nifti1Image at 0x11b411e48>,
 'FA_R': <nibabel.nifti1.Nifti1Image at 0x11b39cbe0>,
 'FP_L': <nibabel.nifti1.Nifti1Image at 0x11b39c198>,
 'FP_R': <nibabel.nifti1.Nifti1Image at 0x11b3845c0>,
 'HCC_roi1_L': <nibabel.nifti1.Nifti1Image at 0x11b40ceb8>,
 'HC

In [6]:
MNI_T2 = dpd.read_mni_template()
MNI_T2_data = MNI_T2.get_data()
MNI_T2_affine = MNI_T2.get_affine()
dwi = img
dwi_affine = dwi.get_affine()
dwi_data = dwi.get_data()
mean_b0 = np.mean(dwi_data[..., gtab.b0s_mask], -1)
warped_b0, mapping = reg.syn_registration(mean_b0, MNI_T2_data,
                                          moving_affine=dwi_affine,
                                          static_affine=MNI_T2_affine,
                                          step_length=0.1,
                                          sigma_diff=2.0,
                                          metric='CC',
                                          dim=3,
                                          level_iters=[10, 10, 5],
                                          prealign=None)



Data size is approximately 35MB
Dataset is already in place. If you want to fetch it again please first remove the folder /Users/arokem/.dipy/mni_template 
Creating scale space from the moving image. Levels: 3. Sigma factor: 0.200000.
Creating scale space from the static image. Levels: 3. Sigma factor: 0.200000.
Optimizing level 2
Optimizing level 1
Optimizing level 0


In [8]:
warped_b0.shape

(197, 233, 189)

In [9]:
mean_b0.shape

(81, 106, 76)

In [11]:
ATR_L1 = afq_templates['ATR_roi1_L']
ATR_L2 = afq_templates['ATR_roi2_L']

In [13]:
ATR_L1_data = ATR_L1.get_data()
ATR_L2_data = ATR_L2.get_data()

In [14]:
ATR_L1_data.dtype

dtype('<i2')

In [15]:
warped_ATR_L1 = mapping.transform_inverse(ATR_L1_data, interpolation='nearest')

In [16]:
warped_ATR_L1.shape

(81, 106, 76)

In [21]:
csd.fit_csd(['/Users/arokem/.dipy/stanford_hardi/HARDI150.nii.gz'], 
            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bval'], 
            ['/Users/arokem/.dipy/stanford_hardi/HARDI150.bvec'], 
            out_dir='.')

/Users/arokem/source/dipy/dipy/reconst/shm.py:367: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m_list = empty(ncoef, 'int')


'./csd_sh_coeff.nii.gz'

In [22]:
localtracking = aft.track('./csd_sh_coeff.nii.gz')

/Users/arokem/source/dipy/dipy/reconst/shm.py:367: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m_list = empty(ncoef, 'int')


In [ ]:
streamlines = list(localtracking)

In [ ]:
## For the corticospinal tract, need to potentially deal with fibers that loop through the brainstem

## Deal with fiber outliers

## Compare with the track probability maps? 